just a project

here we import some necessary libs

In [1]:
import json
import requests
import time

first we get a dict of all heroes, key=id, value=name

In [2]:
def all_heroes():
    all_heroes = {}
    r = requests.get("https://api.opendota.com/api/heroes")
    heroes_raw = json.loads(r.text)
    for hero in heroes_raw:
        all_heroes[hero['id']] = hero['localized_name']
    return all_heroes

In [3]:
match_id = 3432354197

here we form a match_info dict<br/>
it has:
* match_id
* which faction won the match
* list of heroes separated by faction

In [4]:
def full_match_info_dict(match_id):
    '''returns full_match_info_dict from opendota api'''
    r = requests.get("https://api.opendota.com/api/matches/{0}".format(match_id))
    raw_dict = json.loads(r.text)
    if 'error' in raw_dict:
        return None
    else:
        return raw_dict

def avg_mmr(full_match_info_dict):
    '''calculates avg_mmr of a match'''
    ranks = []
    for i in range(10):
        ranks.append(full_match_info_dict['players']
                     [i]['solo_competitive_rank'])
    ranks = list(filter(None.__ne__, ranks))

    try:
        return sum(ranks) / len(ranks)
    except ZeroDivisionError:
        return None

def match_id_parser(full_match_info_dict):
    '''returns match_id of a match'''
    return full_match_info_dict['match_id']

def radiant_won(full_match_info_dict):
    '''returns True if Radiant won the match'''
    return full_match_info_dict['radiant_win']


def match_heroes(full_match_info_dict):
    '''returns a dict that looks like hero_id:is_radiant(bool)'''
    heroes = {}
    for i in range(10):
        heroes[full_match_info_dict['players'][i]['hero_id']] = full_match_info_dict['players'][i]['isRadiant']
    return heroes


def parse_match(match_id):
    '''compiles all info about a match
    input = match_id, output = match_info dict'''
    raw_match_info = full_match_info_dict(match_id)
    match_info = {}
    match_info['id'] = match_id
    
    if raw_match_info is not None:
        match_info['parse_status'] = 'OK'
        match_info['avg_mmr'] = avg_mmr(raw_match_info)
        match_info['radiant_won'] = radiant_won(raw_match_info)
        match_info['heroes'] = match_heroes(raw_match_info)
    else:
        match_info['parse_status'] = 'No match info'
    
    return match_info

here we try to parse several matches

In [5]:
def n_matches_parser(starting_id, n):
    '''returns a dict of a match_info dicts
    opendota guys ask for request rate ≈ 3/sec'''
    parsed_matches = []
    for i in range(n):
        parsed_matches.append(parse_match(starting_id + i))
        time.sleep(0.1)
    return parsed_matches

In [6]:
n_matches_parser(match_id,20)

[{'avg_mmr': 4395.0,
  'heroes': {10: True,
   30: False,
   36: False,
   40: False,
   47: True,
   58: True,
   63: False,
   64: False,
   74: True,
   87: True},
  'id': 3432354197,
  'parse_status': 'OK',
  'radiant_won': True},
 {'avg_mmr': 1168.0,
  'heroes': {14: False,
   20: False,
   23: True,
   28: True,
   36: True,
   38: True,
   52: False,
   97: True,
   103: False,
   104: False},
  'id': 3432354198,
  'parse_status': 'OK',
  'radiant_won': False},
 {'avg_mmr': None,
  'heroes': {1: False,
   28: True,
   29: True,
   47: False,
   49: True,
   71: False,
   92: False,
   101: False,
   106: True,
   108: True},
  'id': 3432354199,
  'parse_status': 'OK',
  'radiant_won': True},
 {'avg_mmr': 2155.5,
  'heroes': {6: True,
   16: True,
   19: True,
   23: True,
   26: False,
   28: False,
   44: False,
   46: False,
   70: True,
   96: False},
  'id': 3432354200,
  'parse_status': 'OK',
  'radiant_won': False},
 {'avg_mmr': 4541.0,
  'heroes': {4: True,
   15: False,


In [33]:
#parse_match(match_id+14)

In [38]:
%%timeit -n 1 -r 1

1
726 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
